In [212]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from dash import Dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import json

Todo:
1. Dropdown menu
2. Borough bar chart (req 1)
3. Neighborhood bar chart (req 1,2)
4. Map of NYC (req 1, 2*/3*)
5. Scatterplots (req 1, 2*/3*)

In [213]:
# load in the data csv files
tree_df = pd.read_csv('./final_data/trees.csv')
borough_df = pd.read_csv('./final_data/boroughs.csv')
neighborhood_df = pd.read_csv('./final_data/UHF.csv')

In [214]:

with open('./geo_data/borough.geo.json', 'r') as file:
    boroughs = json.load(file)
with open('./geo_data/UHF34.geo.json', 'r') as file:
    nbds = json.load(file)
borough_df['BoroName'] = borough_df['Borough']
del nbds["features"][0]

In [215]:
# make the borough graph
fig = px.bar(y=borough_df["Borough"], x=borough_df["Number of Trees"], orientation='h')
fig.update_layout(margin=dict(l=20, r=20, t=20, b=20))
borough_bar = go.Figure(data=fig)

In [216]:
# make the neighborhood graph
fig = px.bar(x=neighborhood_df["UHF34 Neighborhood Name"], y=neighborhood_df["Number of Trees"], orientation='h')
fig.update_layout(margin=dict(l=20, r=20, t=20, b=20))
neighborhood_bar = go.Figure(data=fig)

In [217]:
# TODO: make the map ???
dataname = 'Number of Trees'
fig = go.Figure(data=go.Choropleth(
        geojson=boroughs, # for UHF use `nbds`
        colorscale="Viridis",
        locations=borough_df['BoroName'], #for UHF use `neighborhood_df['UHF34 Code']`
        locationmode="geojson-id",
        featureidkey="properties.BoroName", #for UHF use "properties.UHF" 
        z=borough_df[dataname], # for UHF use `neighborhood_df[dataname]`
        zmin=0,
        zmax=borough_df[dataname].max()
        ))

fig.update_layout(geo = dict(
        landcolor = "rgb(212, 212, 212)",
        resolution = 50,
        lonaxis = dict(
            showgrid = False,
            gridwidth = 0.5,
            range= [ -140.0, -55.0 ],
            dtick = 5
        ),
        lataxis = dict (
            showgrid = False,
            gridwidth = 0.5,
            range= [ 20.0, 60.0 ],
            dtick = 5
        )
    ),
    margin=dict(l=20, r=20, t=20, b=20)
    )

map = go.Figure(data=fig)
map.update_geos(fitbounds="locations")
()


()

In [218]:
# make the scatterplots
fig1 = px.scatter(x=neighborhood_df["Number of Trees"], y=neighborhood_df["Average Fine Particle Pollution"])
fig2 = px.scatter(x=neighborhood_df["Number of Trees"], y=neighborhood_df["Average Diameter"])
fig3 = px.scatter(x=neighborhood_df["Number of Trees"], y=neighborhood_df["Average Summer Surface Temperature"])
fig1.update_layout(margin=dict(l=20, r=20, t=20, b=20))
fig2.update_layout(margin=dict(l=20, r=20, t=20, b=20))
fig3.update_layout(margin=dict(l=20, r=20, t=20, b=20))
metric1_scatter = go.Figure(data=fig1)
metric2_scatter = go.Figure(data=fig2)
metric3_scatter = go.Figure(data=fig3)

neighborhood_df.head()

,Unnamed: 0,Average Status,Average Diameter,Borough,UHF34 Code,Number of Trees,Average Summer Surface Temperature,Average Fine Particle Pollution,Average Ozone Pollution,UHF34 Neighborhood Name
0,0,2.709569,13.385147,Brooklyn,209,14327,94.8,7.671429,31.921429,Bensonhurst - Bay Ridge
1,1,2.690632,14.122453,Queens,404406,36177,97.6,7.600000,31.471429,Bayside Little Neck-Fresh Meadows
2,2,2.705653,11.143644,Brooklyn,203,19973,98.5,8.214286,30.542857,Bedford Stuyvesant - Crown Heights
3,3,2.676278,13.192261,Brooklyn,206,21398,98.6,7.907143,31.264286,Borough Park
4,4,2.653037,14.679529,Brooklyn,208,19100,95.9,7.578571,33.800000,Canarsie - Flatlands


In [219]:
# Dash app setup
app = Dash(__name__) # TODO: add stylesheet
app.layout = html.Div([
    html.Div([
        dcc.Dropdown(
            ['Number of Trees', 'Average Status', 'Average Diameter', 'Average Fine Particle Pollution', 'Average Ozone Pollution', 'Average Summer Surface Temperature'],
            'Number of Trees',
            id='metric-dropdown'
        ),
        html.Div([
            html.Div(
                [dcc.Graph(id='borough-bar', figure=borough_bar),
                    html.Button('Deselect Borough', id='borough-deselect-button')],
                    id='borough-div',
                    className='column'
            ),
            html.Div(
                [ dcc.Graph(id='neighborhood-bar', figure=neighborhood_bar),
                html.Button('Deselect Neighborhood', id='neighborhood-deselect-button')],
                id='neighborhood-div',
                className='column'
            )],
            id='bar-div',
            className='row'
        ),
    ],
    id='far-left-div',
    className='column'
    ),
    html.Div([
        dcc.Graph(id='map', figure=map)
        ],
        id='map-div',
        className='column'
    ),
    html.Div([
            dcc.Graph(id='metric1-scatter', className='scatter', figure=metric1_scatter),
            dcc.Graph(id='metric2-scatter', className='scatter', figure=metric2_scatter),
            dcc.Graph(id='metric3-scatter', className='scatter', figure=metric3_scatter)
        ],
        id='scatter-div',
        className='column'
    )
],
className='row'
)

In [220]:
def update_borough_bar(chosen_metric, clicked_borough):
    
    # Create the bar chart
    fig = px.bar(
        y=borough_df["Borough"], 
        x=borough_df[chosen_metric], 
        title=f'{chosen_metric} by Borough', 
        orientation='h', 
    )

    # Color the clicked borough
    if clicked_borough:
        colors = ['purple' if borough == clicked_borough else '#636EFA' for borough in borough_df["Borough"]]
        fig.update_traces(marker_color=colors)        

    # Update axes and layout
    fig.update_yaxes(title="Borough")
    fig.update_xaxes(title=chosen_metric)
    fig.update_layout(yaxis={'categoryorder':'total ascending'}, margin=dict(l=20, r=20, t=20, b=20))

    borough_bar = go.Figure(data=fig)
    return borough_bar

In [221]:
def update_neighborhood_bar(chosen_metric, clicked_borough, clicked_neighborhood):

    if clicked_borough != None:
        neighborhood_df_filtered = neighborhood_df[neighborhood_df["Borough"] == clicked_borough]
    else:
        neighborhood_df_filtered = neighborhood_df
    
    # TODO: if borough is selected, filter neighborhood_df_filtered by borough
    fig = px.bar(
        y=neighborhood_df_filtered["UHF34 Neighborhood Name"], 
        x=neighborhood_df_filtered[chosen_metric], 
        title=f'{chosen_metric} by Neighborhood', 
        orientation='h', 
    )

    # Color the clicked borough
    if clicked_neighborhood:
        colors = ['purple' if neighborhood == clicked_neighborhood else '#636EFA' for neighborhood in neighborhood_df_filtered["UHF34 Neighborhood Name"]]
        fig.update_traces(marker_color=colors)        

    fig.update_yaxes(title="Neighborhood")
    fig.update_xaxes(title=chosen_metric)
    fig.update_layout(yaxis={'categoryorder':'total ascending'}, margin=dict(l=0, r=0, t=0, b=0))
    neighborhood_bar = go.Figure(data=fig)
    return neighborhood_bar

In [222]:
def update_map(chosen_metric, clicked_borough):
    if clicked_borough == None:
        fig = go.Figure(data=go.Choropleth(
        geojson=boroughs, # for UHF use `nbds`
        colorscale="Viridis",
        locations=borough_df['BoroName'], #for UHF use `neighborhood_df['UHF34 Code']`
        locationmode="geojson-id",
        featureidkey="properties.BoroName", #for UHF use "properties.UHF" 
        z=borough_df[chosen_metric], # for UHF use `neighborhood_df[dataname]`
        zmin=0,
        zmax=borough_df[chosen_metric].max()
        ))

        fig.update_layout(geo = dict(
                landcolor = "rgb(212, 212, 212)",
                resolution = 50,
                lonaxis = dict(
                    showgrid = False,
                    gridwidth = 0.5,
                    range= [ -140.0, -55.0 ],
                    dtick = 5
                ),
                lataxis = dict (
                    showgrid = False,
                    gridwidth = 0.5,
                    range= [ 20.0, 60.0 ],
                    dtick = 5
                )
            ),
            margin=dict(l=20, r=20, t=20, b=20)
            )

        map = go.Figure(data=fig)
        map.update_geos(fitbounds="locations")
        ()
    elif clicked_borough != None:
        fig = go.Figure(data=go.Choropleth(
        geojson=nbds, # for UHF use `nbds`
        colorscale="Viridis",
        locations=neighborhood_df['UHF34 Code'], #for UHF use `neighborhood_df['UHF34 Code']`
        locationmode="geojson-id",
        featureidkey="properties.UHF", #for UHF use "properties.UHF" 
        z=neighborhood_df[chosen_metric], # for UHF use `neighborhood_df[dataname]`
        zmin=0,
        zmax=neighborhood_df[chosen_metric].max()
        ))

        fig.update_layout(geo = dict(
                landcolor = "rgb(212, 212, 212)",
                resolution = 50,
                lonaxis = dict(
                    showgrid = False,
                    gridwidth = 0.5,
                    range= [ -140.0, -55.0 ],
                    dtick = 5
                ),
                lataxis = dict (
                    showgrid = False,
                    gridwidth = 0.5,
                    range= [ 20.0, 60.0 ],
                    dtick = 5
                )
            ),
            margin=dict(l=20, r=20, t=20, b=20)
        )
        map = go.Figure(data=fig)
        if clicked_borough == "Manhattan":
            map.update_geos(
            center=dict(lat=40.7685, lon=-73.9822),  
            projection_scale=175)
        elif clicked_borough =="Brooklyn":
            map.update_geos(
            center=dict(lat=40.6782, lon=-73.9442),  
            projection_scale=175)
        elif clicked_borough =="Bronx":
            map.update_geos(
            center=dict(lat=40.8448, lon=-73.8648),  
            projection_scale=250)
        elif clicked_borough =="Queens":
            map.update_geos(
            center=dict(lat=40.7282, lon=-73.7949),  
            projection_scale=110)
        elif clicked_borough =="Staten Island":
            map.update_geos(
            center=dict(lat=40.5795, lon=-74.1502),  
            projection_scale=240)

    return map

In [223]:
def update_scatters(chosen_metric, clicked_borough, clicked_neighborhood, other_metrics):
    highlighted_df = neighborhood_df[neighborhood_df["Borough"] == clicked_borough] if clicked_borough else None

    # Handle clicked neighborhood logic
    highlighted_neighborhood_df = (
        neighborhood_df[neighborhood_df["UHF34 Neighborhood Name"] == clicked_neighborhood]
        if clicked_neighborhood else None
    )

    scatters = []
    for other_metric in other_metrics:
        if not other_metric or not chosen_metric:
            # Provide a default empty plot
            fig = px.scatter(pd.DataFrame({'x': [], 'y': []}), x='x', y='y')

        fig = px.scatter(
            x=neighborhood_df[chosen_metric],
            y=neighborhood_df[other_metric]
        )
        fig.update_layout(xaxis_title=chosen_metric, yaxis_title=other_metric, margin=dict(l=20, r=20, t=20, b=20))

        # Add highlighted points if a borough is clicked
        if highlighted_df is not None:
            fig.add_scatter(
                x=highlighted_df[chosen_metric],
                y=highlighted_df[other_metric],
                mode='markers',
                marker=dict(color='red'),
                name=f"{clicked_borough}"
            )
        
        # Add highlighted points if a neighborhood is clicked
        if highlighted_neighborhood_df is not None:
            fig.add_scatter(
                x=highlighted_neighborhood_df[chosen_metric],
                y=highlighted_neighborhood_df[other_metric],
                mode='markers',
                marker=dict(color='green'),
                name=f"{clicked_neighborhood}"
            )
        
        # Update layout and return the plot
        scatters.append(fig)
    return scatters



In [224]:
@app.callback(
    Output("borough-bar", "figure"),
    Output('borough-bar', 'clickData'),
    Output('borough-deselect-button', 'n_clicks'),
    Output("neighborhood-bar", "figure"),
    Output('neighborhood-bar', 'clickData'),
    Output("neighborhood-deselect-button", 'n_clicks'),
    Output("map", "figure"),
    Output("metric1-scatter", "figure"),
    Output("metric2-scatter", "figure"),
    Output("metric3-scatter", "figure"),
    [
        Input("metric-dropdown", "value"),
        Input('borough-bar', 'clickData'),
        Input('borough-deselect-button', 'n_clicks'),
        Input('neighborhood-bar', 'clickData'),
        Input('neighborhood-deselect-button', 'n_clicks')
    ]
)
def update_dash(chosen_metric, clicked_borough_bar, borough_deselect_clicks, clicked_nbd_bar, nbd_deselect_clicks):
    # If the borough is clicked, store the clicked borough
    borough_clickdata = clicked_borough_bar
    clicked_borough = borough_clickdata['points'][0]['y'] if borough_clickdata else None
    if borough_deselect_clicks and borough_deselect_clicks > 0:
         clicked_borough = None
         borough_clickdata = None
    
    nbd_clickdata = clicked_nbd_bar
    clicked_nbd = nbd_clickdata['points'][0]['y'] if nbd_clickdata else None
    if nbd_deselect_clicks and nbd_deselect_clicks > 0:
         clicked_nbd = None
         nbd_clickdata = None         

    borough_bar = update_borough_bar(chosen_metric, clicked_borough)
    neighborhood_bar = update_neighborhood_bar(chosen_metric, clicked_borough, clicked_nbd)
    map = update_map(chosen_metric, clicked_borough)

    tree_metrics = ['Number of Trees', 'Average Status', 'Average Diameter']
    env_metrics = ['Average Fine Particle Pollution', 'Average Ozone Pollution', 'Average Summer Surface Temperature']

    other_metrics = env_metrics if chosen_metric in tree_metrics else tree_metrics

    [metric1_scatter, metric2_scatter, metric3_scatter] = update_scatters(chosen_metric, clicked_borough, clicked_nbd, other_metrics)

    return borough_bar, borough_clickdata, None, neighborhood_bar, nbd_clickdata, None, map, metric1_scatter, metric2_scatter, metric3_scatter

if __name__ == '__main__':
    app.run_server(debug=True)